# Tuning Hyperparameters

There are many machine learning algorithms that require *hyperparameters* (parameter values that influence training, but can't be determined from the training data itself). For example, when training a logistic regression model, you can use a *regularization rate* hyperparameter to counteract bias in the model; or when training a convolutional neural network, you can use hyperparameters like *learning rate* and *batch size* to control how weights are adjusted and how many data items are processed in a mini-batch respectively. The choice of hyperparameter values can significantly affect the performance of a trained model, or the time taken to train it; and often you need to try multiple combinations to find the optimal solution.

In this case, you'll use a simple example of a logistic regression model with a single hyperparameter, but the principles apply to any kind of model you can train with Azure Machine Learning.

## Connect to Your Workspace

The first thing you need to do is to connect to your workspace using the Azure ML SDK.

> **Note**: You may be prompted to authenticate. Just copy the code and click the link provided to sign into your Azure subscription, and then return to this notebook.

In [26]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.13.0 to work with customer_360_ws


## Create a Training Script

You're going to use a Python script to train a machine learning model based on the flight_delays data, so let's start by creating a folder for the script and data files.

In [27]:
import os

experiment_folder = 'flight_delays_hyperdrive'
os.makedirs(experiment_folder, exist_ok=True)

print('Folder ready.')

Folder ready.


In [40]:
!pip install --upgrade azureml-sdk[notebooks,automl,explain]
!pip install --upgrade azureml-interpret

Requirement already up-to-date: azureml-sdk[automl,explain,notebooks] in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.13.0)
Requirement already up-to-date: azureml-interpret in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.13.0)


### Interpreting Models

You can use Azure Machine Learning to interpret a model by using an *explainer* that quantifies the amount of influence each feature contribues to the predicted label. There are many common explainers, each suitable for different kinds of modeling algorithm; but the basic approach to using them is the same.

Run the following cell to ensure that you have the latest version of the Azure ML SDK installed, and in addition install the Azure ML Interpretability library. You can use this to interpret many typical kinds of model, even if they haven't been trained in an Azure ML experiment or registered in an Azure ML workspace.


Now create the Python script to train the model. This must include:

- A parameter for each hyperparameter you want to optimize (in this case, there's only the regularization hyperparameter)
- Code to log the performance metric you want to optimize for (in this case, you'll log both AUC and accuracy, so you can choose to optimize the model for either of these)

In [28]:
%%writefile $experiment_folder/flight_delays_training.py
# Import libraries
import argparse
import json
import joblib
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Import libraries for model explanation
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient
from interpret.ext.blackbox import TabularExplainer


# Set regularization parameter
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate




# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")


dataset = run.input_datasets['flight_delays_data'].to_pandas_dataframe().dropna() # Get the training data from the estimator input

# Remove target leaker and features that are not useful
target_leakers = ['DepDel15','ArrDelay','Cancelled','Year']
dataset.drop(columns=target_leakers, axis=1, inplace=True)

# convert some variables to categorical features
columns_as_categorical = ['OriginAirportID','DestAirportID','ArrDel15']
dataset[columns_as_categorical] = dataset[columns_as_categorical].astype('object')

categorical_feature_mask = dataset.dtypes == object 
categorical_cols = dataset.columns[categorical_feature_mask].tolist()

le = LabelEncoder()
dataset[categorical_cols] = dataset[categorical_cols].apply(lambda col:le.fit_transform(col))
dataset = dataset.dropna()

features = ['Month', 'DayofMonth', 'DayOfWeek', 'Carrier', 'OriginAirportID', 'DestAirportID', 'CRSDepTime', 'DepDelay', 'CRSArrTime']
labels = ['no-delay', 'delay']

# Separate features and labels
X, y = dataset[features].values, dataset['ArrDel15'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))


# work on confusion matrix

cnf_matrix = confusion_matrix(y_test, y_hat)

cnf_matrix_list = cnf_matrix.tolist()
cnf_matrix_json_ = json.dumps(cnf_matrix_list)
run.log_confusion_matrix(name='Confusion_matrix', value=cnf_matrix_json_)

class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.show()
run.log_image(name='confusion_matrix_img', plot=fig)


os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/flight_delays_hyperdrive_model.pkl')

# Get explanation
explainer = TabularExplainer(model, X_train, features=features, classes=labels)
explanation = explainer.explain_global(X_test)

# Get an Explanation Client and upload the explanation
explain_client = ExplanationClient.from_run(run)
explain_client.upload_model_explanation(explanation, comment='Tabular Explanation')


run.complete()

Overwriting flight_delays_hyperdrive/flight_delays_training.py


## Define an Environment

Now you can run the experiment, using an estimator to run the training script. Note that the **azureml-interpret** library is included in the training environment so the script can create a **TabularExplainer**, and the **azureml-contrib-interpret** package is included so the script can use the **ExplainerClient** class.

In [30]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create a Python environment for the experiment
flight_delays_hyperdrive_env = Environment("flight-delays-hyperdrive-experiment-env")
flight_delays_hyperdrive_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
flight_delays_hyperdrive_env.docker.enabled = True # Use a docker container

# Create a set of package dependencies (conda or pip as required)
flight_delays_hyperdrive_packages = CondaDependencies.create(conda_packages=['scikit-learn', 'pandas'],
                                          pip_packages=['azureml-defaults', 
                                          'azureml-dataprep[pandas]', 
                                          'azureml-interpret',
                                          'azureml-contrib-interpret',
                                          'matplotlib', 
                                          'seaborn'])

# Add the dependencies to the environment
flight_delays_hyperdrive_env.python.conda_dependencies = flight_delays_hyperdrive_packages

print(flight_delays_hyperdrive_env.name, 'defined.')

# Register the environment
flight_delays_hyperdrive_env.register(workspace=ws)
print(flight_delays_hyperdrive_env.name, 'registered.')

flight-delays-hyperdrive-experiment-env defined.
flight-delays-hyperdrive-experiment-env registered.


## Run a *Hyperdrive* Experiment

Azure Machine Learning includes a hyperparameter tuning capability through *Hyperdrive* experiments. These experiments launch multiple child runs, each with a different hyperparameter combination. The run producing the best model (as determined by the logged target performance metric for which you want to optimize) can be identified, and its trained model selected for registration and deployment.

In [31]:
from azureml.core import Experiment, Environment
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive import GridParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.widgets import RunDetails


# Get the environment
registered_env = Environment.get(ws, 'flight-delays-hyperdrive-experiment-env')

# specify cluster name
cluster_name = "aml-cluster"

# Get the training dataset
flight_delays_ds = ws.datasets.get("flight_delays_data")


# Sample a range of parameter values
params = GridParameterSampling(
    {
        # There's only one parameter, so grid sampling will try each value - with multiple parameters it would try every combination
        '--regularization': choice(0.001, 0.005, 0.01, 0.05, 0.1, 1.0)
    }
)


# Create an estimator that uses the remote compute
hyper_estimator = SKLearn(source_directory=experiment_folder,
                          inputs=[flight_delays_ds.as_named_input('flight_delays_data')], # Pass the dataset as an input...
                          environment_definition = registered_env,
                        #   pip_packages=['azureml-sdk'], # ...so we need azureml-dataprep (it's in the SDK!)
                          entry_script='flight_delays_training.py',
                          compute_target = cluster_name,)

# Configure hyperdrive settings
hyperdrive = HyperDriveConfig(estimator=hyper_estimator, 
                          hyperparameter_sampling=params, 
                          policy=None, 
                          primary_metric_name='AUC', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6,
                          max_concurrent_runs=4)

# Run the experiment
experiment = Experiment(workspace = ws, name = 'flight-delays-training-hyperdrive')
run = experiment.submit(config=hyperdrive)

# Show the status in the notebook as the experiment runs
RunDetails(run).show()
run.wait_for_completion()

WARNING - If environment_definition or conda_dependencies_file_path is specified, Azure ML will not install any framework related packages on behalf of the user.
WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

You can view the experiment run status in the widget above. You can also view the main Hyperdrive experiment run and its child runs in [Azure Machine Learning studio](https://ml.azure.com).

> **Note**: The widget may not refresh. You'll see summary information displayed below the widget when the run has completed.

## Determine the Best Performing Run

When all of the runs have finished, you can find the best one based on the performance metric you specified (in this case, the one with the best AUC).

In [37]:
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)

best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details() ['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print(' -AUC:', best_run_metrics['AUC'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Regularization Rate:',parameter_values)

{'run_id': 'HD_b6428d8d-7ba3-4f9d-8b0b-f94a2de866e9_3', 'hyperparameters': '{"--regularization": 0.05}', 'best_primary_metric': 0.9182643378898959, 'status': 'Completed'}
{'run_id': 'HD_b6428d8d-7ba3-4f9d-8b0b-f94a2de866e9_4', 'hyperparameters': '{"--regularization": 0.1}', 'best_primary_metric': 0.918264206659151, 'status': 'Completed'}
{'run_id': 'HD_b6428d8d-7ba3-4f9d-8b0b-f94a2de866e9_1', 'hyperparameters': '{"--regularization": 0.005}', 'best_primary_metric': 0.9182639843008541, 'status': 'Completed'}
{'run_id': 'HD_b6428d8d-7ba3-4f9d-8b0b-f94a2de866e9_5', 'hyperparameters': '{"--regularization": 1.0}', 'best_primary_metric': 0.9182638502886536, 'status': 'Completed'}
{'run_id': 'HD_b6428d8d-7ba3-4f9d-8b0b-f94a2de866e9_0', 'hyperparameters': '{"--regularization": 0.001}', 'best_primary_metric': 0.9182636489447646, 'status': 'Completed'}
{'run_id': 'HD_b6428d8d-7ba3-4f9d-8b0b-f94a2de866e9_2', 'hyperparameters': '{"--regularization": 0.01}', 'best_primary_metric': 0.9182621252280925

Now that you've found the best run, you can register the model it trained.

In [38]:
from azureml.core import Model

# Register the model
best_run.register_model(model_path='outputs/flight_delays_hyperdrive_model.pkl', model_name='flight_delays_hyperdrive_model',
                   tags={'Training context':'Hyperdrive Flight delays model'},
                   properties={'AUC': best_run.get_metrics()['AUC'], 'Accuracy': best_run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

flight_delays_hyperdrive_model version: 4
	 Training context : Hyperdrive Flight delays model
	 AUC : 0.9182643378898959
	 Accuracy : 0.9200313213961809


flight_delays_hyperdrive_model version: 3
	 Training context : Hyperdrive Flight delays model
	 AUC : 0.9182643378898959
	 Accuracy : 0.9200313213961809


flight_delays_hyperdrive_model version: 2
	 Training context : Hyperdrive Flight delays model
	 AUC : 0.9182643378898959
	 Accuracy : 0.9200313213961809


flight_delays_hyperdrive_model version: 1
	 Training context : Hyperdrive Flight delays model
	 AUC : 0.9182638145390434
	 Accuracy : 0.9200263654790637


flight_delays_model version: 4
	 Training context : Parameterized SKLearn Estimator
	 AUC : 0.918264206659151
	 Accuracy : 0.9200276044583431


flight_delays_model version: 3
	 Training context : Parameterized SKLearn Estimator
	 AUC : 0.918264206659151
	 Accuracy : 0.9200276044583431


diabetes_model version: 9
	 Training context : Inline Training
	 AUC : 0.8788546693865366
	 

## Retrieve the Feature Importance Values

With the experiment run completed, you can use the **ExplanationClient** class to retrieve the feature importance from the explanation registered for the run.

In [39]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# Get the feature explanations
client = ExplanationClient.from_run(run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

ExplanationNotFoundException: ExplanationNotFoundException:
	Message: Did not find any explanations matching comment or raw filters.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "inner_error": {
            "code": "NotFound"
        },
        "message": "Did not find any explanations matching comment or raw filters."
    }
}